<a href="https://colab.research.google.com/github/oytungunes/ResearchBilkent/blob/main/Peoplewalking_DCNN_twoclass_noaugment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
# Burada sadece range-doppler ve spectrogram datası encoder decoder yapısı ile  Bi-LSTM ile train edildi.
# augmentation kaldırıldı
#[0.7719367623329163, 0.8422924757003785, 0.8249011754989624]
#Mean test accuracy is 0.813, max test accuracy is 0.842, min test accuracy is 0.772,  std of test accuracy is 0.030.
#Time elapsed through all process: 184.902, sec



In [23]:
from keras.models import Sequential
from tensorflow.keras import layers
from keras.models import Model
from keras.layers import Activation, Dropout, Flatten, Dense, BatchNormalization, Normalization, Input, Conv2D, MaxPooling2D, Concatenate, GRU, LSTM, GRU, TimeDistributed, Bidirectional
import tensorflow as tf
import scipy.io
import numpy as np
import cv2
import matplotlib.pyplot as plt
import random
from numpy.random import seed
from sklearn.model_selection import KFold, StratifiedKFold
import time
from sklearn.metrics import precision_recall_fscore_support
from keras.callbacks import EarlyStopping
from numpy import array
from numpy import argmax
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import OneHotEncoder
from keras import backend as K
import gc
import os
from keras.layers import LeakyReLU

In [24]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [25]:



# Load Spectrogram fast data
spectrogram_fast_rsdcnn = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_rsdcnn.mat')
spectrogram_fast_rsdcnn = spectrogram_fast_rsdcnn['spectrogram_fast_rsdcnn']
spectrogram_fast_rsdcnn = np.transpose(spectrogram_fast_rsdcnn, (2, 0, 1)) # label x width x height
spectrogram_fast_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_fast_label.mat')
spectrogram_fast_label = spectrogram_fast_label['spectrogram_fast_label']


# Load Spectrogram slow data
spectrogram_slow_rsdcnn= scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_rsdcnn.mat')
spectrogram_slow_rsdcnn = spectrogram_slow_rsdcnn['spectrogram_slow_rsdcnn']
spectrogram_slow_rsdcnn = np.transpose(spectrogram_slow_rsdcnn, (2, 0, 1)) # label x width x height
spectrogram_slow_label = scipy.io.loadmat('/content/drive/MyDrive/dataset_peoplewalking/spectrogram_slow_label.mat')
spectrogram_slow_label = spectrogram_slow_label['spectrogram_slow_label']



In [44]:
spectrogram_concat_label = np.concatenate((spectrogram_fast_label,spectrogram_slow_label),axis=0)

# One-Hot Encode Label Spectrogram
# integer encode
label_encoder = LabelEncoder()
integer_encoded = label_encoder.fit_transform(spectrogram_concat_label)
# binary encode
onehot_encoder = OneHotEncoder(sparse_output=False)
integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
spectrogram_concat_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)



/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [45]:
# Concat spectrogram data

spectrogram_concat = np.concatenate((spectrogram_fast_rsdcnn,spectrogram_slow_rsdcnn),axis=0)
spectrogram_concat = spectrogram_concat[:,:,:,np.newaxis]

# Shuffle concat spectrogram
shuffle_indx = random.sample(range(0, spectrogram_concat.shape[0]), spectrogram_concat.shape[0]) # split validation data

spectrogram_concat_shuffle = spectrogram_concat[shuffle_indx,:,:,:]
spectrogram_concat_label_shuffle_onehotencoded = spectrogram_concat_label_onehotencoded[shuffle_indx,:]
spectrogram_concat_label_shuffle = spectrogram_concat_label[shuffle_indx,:]




In [46]:
# ---------------- Augmente and shuffle (train and test) data data ----------------
data_augmentation = tf.keras.Sequential([
  layers.RandomFlip("horizontal_and_vertical"),
  layers.RandomRotation(0.1),
])

def split_and_augmentation_of_training(spectrogram_concat_shuffle_train,spectrogram_concat_label_shuffle):
  # ---------------- Parameters ----------------

  dummy_label = np.zeros((spectrogram_concat_shuffle_train.shape[0],1))
  for randomlist_for_train_indx, randomlist_for_validation_indx in kfold.split(spectrogram_concat_shuffle_train,dummy_label):
    randomlist_for_validation_indx
  # get validation data
  spectrogram_validation_data = spectrogram_concat_shuffle_train[randomlist_for_validation_indx,:,:,:]
  spectrogram_validation_labels = spectrogram_concat_label_shuffle[randomlist_for_validation_indx,:]
  # get training data
  spectrogram_training_data = spectrogram_concat_shuffle_train[randomlist_for_train_indx,:,:,:]
  spectrogram_training_labels = spectrogram_concat_label_shuffle_train[randomlist_for_train_indx,:]

  # Rotate Augmentation
  # get slow and fast indexes of training data
  slow_indexes = np.where(spectrogram_training_labels == 0)[0]
  fast_indexes = np.delete(range(0, spectrogram_training_labels.shape[0]), slow_indexes)

  slow_spectrograms_train = spectrogram_training_data[slow_indexes,:,:,:]
  size_of_samples_slow = slow_spectrograms_train.shape[0]

  fast_spectrograms_train = spectrogram_training_data[fast_indexes,:,:,:]
  size_of_samples_fast = fast_spectrograms_train.shape[0]


  augmented_image_fast = fast_spectrograms_train
  augmented_image_slow = slow_spectrograms_train
  spectrograms_fast_label = np.ones((size_of_samples_fast,1))
  spectrograms_slow_label = np.zeros((size_of_samples_slow,1))

  spectrogram_training_data = np.concatenate((augmented_image_fast,augmented_image_slow),axis=0)
  spectrogram_training_labels = np.concatenate((spectrograms_fast_label,spectrograms_slow_label),axis=0)


  return (spectrogram_training_data,spectrogram_training_labels,\
     spectrogram_validation_data, spectrogram_validation_labels)


In [47]:
def normalize_inputs(range_doppler_concat_shuffle, normalize_inputs_enable):
  # ---------------- Normalize Inputs ----------------
  if normalize_inputs_enable == True:
    layer = Normalization(axis=None)
    layer.adapt(range_doppler_concat_shuffle)
    range_doppler_concat_shuffle = layer(range_doppler_concat_shuffle)
  else:
    range_doppler_concat_shuffle = range_doppler_concat_shuffle
  return(range_doppler_concat_shuffle)


normalize_inputs_enable = 1
spectrogram_concat_shuffle = normalize_inputs(spectrogram_concat_shuffle, normalize_inputs_enable)
spectrogram_concat_shuffle = np.float32(spectrogram_concat_shuffle)


In [48]:

spectrogram_concat_shuffle = np.transpose(spectrogram_concat_shuffle, axes = (0,2,1,3))

In [53]:
print(spectrogram_concat.shape)
print(spectrogram_concat_label_shuffle.shape)

(114, 64, 32, 1)
(114, 1)


In [59]:
t = time.time()
# ---------- Parameters ----------------
normalize_inputs_enable = True
num_folds = 5
kfold = StratifiedKFold(n_splits=num_folds, shuffle=True, random_state = None) # random_state = 1 ile split run'dan run'a sabit.
test_accuracy_per_run = []
f1_score_per_run = []
epoch_number = 120
batch_size = 32
dense_size = 64
dropout_prob_dense = 0.5
repeat_of_mixup = 5
number_of_repeat = 3
unit_number_of_lstm = 16
dense_unit_of_range_doppler_function = 256
dense_unit_of_spectrogram_function = 8
decoder_dense_unit = 256
for repeat_run_number in range(number_of_repeat):
  test_accuracy_per_fold = []
  f1_score_per_fold = []
  if repeat_run_number > 0:
    del spectrogram_concat_shuffle_test
    del spectrogram_concat_shuffle_train
    del spectrogram_train_data

  for randomlist_for_train_indx, randomlist_for_test_indx in kfold.split(spectrogram_concat_shuffle,spectrogram_concat_label_shuffle):
    gc.collect()
    K.clear_session()

    # test data
    spectrogram_concat_shuffle_test = spectrogram_concat_shuffle[randomlist_for_test_indx,:,:,:]
    spectrogram_concat_label_shuffle_test = spectrogram_concat_label_shuffle[randomlist_for_test_indx,:]
    #train data
    spectrogram_concat_shuffle_train = spectrogram_concat_shuffle[randomlist_for_train_indx,:,:,:]
    spectrogram_concat_label_shuffle_train = spectrogram_concat_label_shuffle[randomlist_for_train_indx,:]
      # ---------------- MixUp Augmentation ----------------
    (spectrogram_train_data,spectrogram_train_label,\
     spectrogram_validation_data, spectrogram_validation_labels)  =\
      split_and_augmentation_of_training(spectrogram_concat_shuffle_train,spectrogram_concat_label_shuffle_train)



    # One-Hot Encode Label Train
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_train_label)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    training_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)

        # One-Hot Encode Label Validation
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_validation_labels)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    validation_labels_onehotencoded = onehot_encoder.fit_transform(integer_encoded)


       # One-Hot Encode Label Test
    # integer encode
    label_encoder = LabelEncoder()
    integer_encoded = label_encoder.fit_transform(spectrogram_concat_label_shuffle_test)
    # binary encode
    onehot_encoder = OneHotEncoder(sparse_output=False)
    integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
    test_label_onehotencoded = onehot_encoder.fit_transform(integer_encoded)





    # ---------------- Neural Network Architecture ----------------

    model = Sequential()
    model.add(layers.Conv2D(32, (3, 3),input_shape=(32,64,1)))
    model.add(Dropout(0.2))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(8, kernel_size=(3, 3)))
    model.add(Dropout(0.2))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Conv2D(16, kernel_size=(3, 3)))
    model.add(Dropout(0.2))
    model.add(layers.MaxPooling2D((2, 2)))
    model.add(layers.Flatten())
    model.add(layers.Dense(2, activation='softmax'))
    opt = tf.keras.optimizers.Adam(0.0001)
    model.summary()


    # ---------------- Compile and Fit ----------------
    model.compile(loss='categorical_crossentropy',
                  optimizer='adam',
                  metrics=['accuracy'])

    earlyStopping = EarlyStopping(monitor='val_loss', patience=15, verbose=0,restore_best_weights=True, mode='min')

    print(spectrogram_train_data.shape)
    print(training_labels_onehotencoded.shape)



    history = model.fit(( spectrogram_train_data),(training_labels_onehotencoded),
                    epochs=epoch_number,
                    batch_size=batch_size,
                    shuffle = True,
                    callbacks=[earlyStopping],
                    validation_data = ((spectrogram_validation_data) , (validation_labels_onehotencoded)))
    test_loss, test_accuracy  = model.evaluate([spectrogram_concat_shuffle_test],\
                                               [test_label_onehotencoded],
                  batch_size=batch_size)
    gc.collect()

    test_accuracy_per_fold.append(test_accuracy)
    del model
  test_accuracy_per_run.append(sum(test_accuracy_per_fold)/num_folds)
  print(test_accuracy_per_run)
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)},\
 max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))},\
 min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, \
 std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}.')
elapsed = time.time() - t
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')

/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/sklearn/preprocessing/_label.py:110: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:113: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer u

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ conv2d (Conv2D)                 │ (None, 30, 62, 32)     │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 30, 62, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 15, 31, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 13, 29, 8)      │         2,312 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_1 (Dropout)             │ (None, 13, 29, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 6, 14, 8)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 4, 12, 16)      │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_2 (Dropout)             │ (None, 4, 12, 16)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 2, 6, 16)       │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 192)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 2)              │           386 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 4,186 (16.35 KB)

 Trainable params: 4,186 (16.35 KB)

...

 Trainable params: 4,186 (16.35 KB)

 Non-trainable params: 0 (0.00 B)

(74, 32, 64, 1)
(74, 2)
Epoch 1/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 7s 1s/step - accuracy: 0.5622 - loss: 0.7757 - val_accuracy: 0.5000 - val_loss: 0.6629
Epoch 2/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 31ms/step - accuracy: 0.6507 - loss: 0.6207 - val_accuracy: 0.7778 - val_loss: 0.5797
Epoch 3/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8379 - loss: 0.5419 - val_accuracy: 0.7222 - val_loss: 0.5257
Epoch 4/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8319 - loss: 0.4899 - val_accuracy: 0.7222 - val_loss: 0.5038
Epoch 5/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8067 - loss: 0.4898 - val_accuracy: 0.7222 - val_loss: 0.5036
Epoch 6/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 29ms/step - accuracy: 0.8340 - loss: 0.4355 - val_accuracy: 0.7222 - val_loss: 0.5269
Epoch 7/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 27ms/step - accuracy: 0.8127 - loss: 0.4176 - val_accuracy: 0.6667 - val_loss: 0.5992
Epoch 8/120
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 30ms/step - accuracy: 0.8564 - loss: 0.4074 - val_accura

/usr/local/lib/python3.11/dist-packages/keras/src/models/functional.py:241: UserWarning: The structure of `inputs` doesn't match the expected structure.
Expected: keras_tensor
Received: inputs=('Tensor(shape=(None, 32, 64, 1))',)
  warnings.warn(msg)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 477ms/step - accuracy: 0.8636 - loss: 0.3782
[0.7719367623329163, 0.8422924757003785, 0.8249011754989624]
Mean test accuracy is 0.813, max test accuracy is 0.842, min test accuracy is 0.772,  std of test accuracy is 0.030.
Time elapsed through all process: 184.902, sec


In [ ]:
print(f'Mean test accuracy is {"{:.3f}".format(sum(test_accuracy_per_run)/number_of_repeat)}, mean test f1 score is {"{:.3f}".format(sum(f1_score_per_run)/number_of_repeat)}, \
max test accuracy is {"{:.3f}".format(max(test_accuracy_per_run))}, max test f1 score is {"{:.3f}".format(max(f1_score_per_run))}, \
min test accuracy is {"{:.3f}".format(min(test_accuracy_per_run))}, min test f1 score is {"{:.3f}".format(min(f1_score_per_run))}, \
std of test accuracy is {"{:.3f}".format(np.std(test_accuracy_per_run, axis=0))}, std of test f1 score is {"{:.3f}".format(np.std(f1_score_per_run, axis=0))}')
print(f'Time elapsed through all process: {"{:.3f}".format(elapsed)}, sec')